Set up OS

In [ ]:
!pip install google-auth google-auth-oauthlib google-auth-httplib2
!pip install google-api-python-client
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import os
os.chdir('/content/drive/My Drive')

# Load Panel Data

In [2]:
df = pd.read_csv("/content/drive/MyDrive/Min_Charecteristics_Tilburg_1963_ALLSAMPLE.csv")

# Imports

In [3]:
!pip install torch_geometric
import pandas as pd
import numpy as np
import torch
import os
from torch_geometric.data import Data


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.7 MB/s eta 0:00:00


# Normalize data And lag feutures

In [4]:
df['Mom1m'] = df['excess_return']
df['Y_excess_return'] = df['excess_return']
# Remove duplicates while keeping the last occurrence
df = df.sort_values('date').drop_duplicates(subset=['date', 'PERMNO'], keep='last')




df = df.sort_values(by=['PERMNO', 'date'])
# Shift the excess return by one month within each firm

# List of column names to lag
factor_columns = [
    "Beta", "RoE", "InvestPPEInv", "ShareIss5Y", "Accruals", "dNoa",
    "GP", "AssetGrowth",  "Investment", "market_cap_adjusted", "excess_return",
    "BM", "CompEquIss", "OperProf",  "MaxRet", "IndMom", "DolVol" ,"Mom1m" , "Mom6m", "Mom12m"
]


# Loop through each column in the list and shift them forward by one period within each group
for column in factor_columns:
    df[column] = df.groupby('PERMNO')[column].shift(1)




# List of column names to standardize
factor_columns = [
    "Beta", "RoE", "InvestPPEInv", "ShareIss5Y", "Accruals", "dNoa",
    "GP", "AssetGrowth",  "Investment", "market_cap_adjusted",
    "BM", "CompEquIss", "OperProf",  "MaxRet", "IndMom", "DolVol" ,"Mom1m" , "Mom6m", "Mom12m"
]
# Compute the mean and standard deviation for each factor column grouped by 'date'
means = df.groupby('date')[factor_columns].transform('mean')
stds = df.groupby('date')[factor_columns].transform('std')

# Standardize the existing columns without creating new ones
for column in factor_columns:
    df[column] = (df[column] - means[column]) / stds[column]


df = df.dropna()

# Convert panel data to graph data for each month

# Using Absulute Values For Correlation

Function construct_graph_with_correlation_sign to construct the graphs based on the Pearson correlation on historical returns. Only alowing for absolute values.

In [6]:

def construct_graph_with_absolute_correlation(df, factor_columns, return_column, W, theta):

    original_dates = pd.to_datetime(df['date'])

    # Dynamic directory name based on W and theta
    save_dir = f'Corrsaved(absolute)_graphs_W{W}_theta{theta}'
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)


    df['formatted_date'] = original_dates.dt.strftime('%Y%m%d').astype(int)
    unique_dates = sorted(df['formatted_date'].unique())

    # Loop over each datre starting from the Wth date to ensure a full window
    for i in range(W - 1, len(unique_dates)):
        current_date = unique_dates[i]
        window_dates = unique_dates[i - W + 1:i + 1]
        window_df = df[df['formatted_date'].isin(window_dates)]

        if 'PERMNO' not in window_df.columns or return_column not in window_df.columns:
            continue  # Skip this iteration if required columns are missing

        pivot_df = window_df.pivot(index='formatted_date', columns='PERMNO', values=return_column)
        correlation_matrix = pivot_df.corr().values

        abs_corr = np.abs(correlation_matrix)
        edge_indices = np.where((abs_corr > theta) & (np.eye(len(df['PERMNO'].unique()), dtype=bool) == False))

        edge_index = torch.tensor(np.array(edge_indices), dtype=torch.long)
        edge_weights = torch.tensor(abs_corr[edge_indices], dtype=torch.float)

        latest_df = df[df['formatted_date'] == current_date]
        if 'PERMNO' in latest_df.columns:
            latest_attributes = latest_df[['PERMNO'] + factor_columns].set_index('PERMNO').reindex(df['PERMNO'].unique())
            node_features_tensor = torch.tensor(latest_attributes.drop(columns='PERMNO', errors='ignore').values, dtype=torch.float)
            permno_tensor = torch.tensor(latest_attributes.index.values, dtype=torch.int)

            # Ensure 'date' is also saved as a tensor
            date_tensor = torch.tensor([current_date] * len(latest_attributes), dtype=torch.long)

            graph_data = Data(x=node_features_tensor, edge_index=edge_index, edge_attr=edge_weights, permno=permno_tensor, date=date_tensor)
            filename = f"CorrGraph_{current_date}_W{W}_theta{theta}.pt"
            filepath = os.path.join(save_dir, filename)
            torch.save(graph_data, filepath)
            print(f"Graph for {current_date} saved to {filepath}")
            print(f"Graph for {current_date}: {len(df['PERMNO'].unique())} nodes, {edge_index.size(1) // 2} edges")
        else:
            print(f"Missing 'PERMNO' in latest data for {current_date}")

    # Cleanup to avoid confusion
    df.drop('formatted_date', axis=1, inplace=True)

factor_columns =[
    "Beta", "RoE", "InvestPPEInv", "ShareIss5Y", "Accruals", "dNoa",
    "GP", "AssetGrowth",  "Investment", "market_cap_adjusted",
    "BM", "CompEquIss", "OperProf",  "MaxRet", "IndMom", "DolVol" ,"Mom1m" , "Mom6m", "Mom12m", "Y_excess_return"]

# Define the return column
return_column = "excess_return"


Call the Function

In [ ]:

# usage
Ws = [12, 24, 36, 48, 60]
thetas = [0.00001, 0.1, 0.2, 0.3, 0.4, 0.5 , 0.9]
for W in Ws:
    for theta in thetas:
        construct_graph_with_absolute_correlation(df, factor_columns, return_column, W, theta)


# Pearson Correlation wich allows for both negative and positive edges

Function for pearson correlation: is defined here: construct_graph_with_correlation_sign


In [ ]:
import os
import pandas as pd
import numpy as np
import torch
from torch_geometric.data import Data

def construct_graph_with_correlation_sign(df, factor_columns, return_column, W, theta):
    # Store the original date format
    original_dates = pd.to_datetime(df['date'])

    # Dynamic directory name based on W and theta
    save_dir = f'Corrsaved(newBest)_graphs_W{W}_theta{theta}'
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # Convert 'date' column to integer format
    df['formatted_date'] = original_dates.dt.strftime('%Y%m%d').astype(int)
    unique_dates = sorted(df['formatted_date'].unique())

    # Loop over each date starting from the Wth date to ensure a full window
    for i in range(W - 1, len(unique_dates)):
        current_date = unique_dates[i]
        window_dates = unique_dates[i - W + 1:i + 1]
        window_df = df[df['formatted_date'].isin(window_dates)]

        # Skip this iteration if required columns are missing
        if 'PERMNO' not in window_df.columns or return_column not in window_df.columns:
            continue

        # Create pivot table for the return column
        pivot_df = window_df.pivot(index='formatted_date', columns='PERMNO', values=return_column)
        correlation_matrix = pivot_df.corr().values

        # Get absolute values of the correlation matrix
        abs_corr = np.abs(correlation_matrix)
        edge_indices = np.where((abs_corr > theta) & (np.eye(len(df['PERMNO'].unique()), dtype=bool) == False))

        # Convert edge indices and weights to tensors
        edge_index = torch.tensor(np.array(edge_indices), dtype=torch.long)
        edge_weights = torch.tensor(correlation_matrix[edge_indices], dtype=torch.float)

        # Get the latest data for the current date
        latest_df = df[df['formatted_date'] == current_date]
        if 'PERMNO' in latest_df.columns:
            # Prepare node features tensor
            latest_attributes = latest_df[['PERMNO'] + factor_columns].set_index('PERMNO').reindex(df['PERMNO'].unique())
            node_features_tensor = torch.tensor(latest_attributes.drop(columns='PERMNO', errors='ignore').values, dtype=torch.float)
            permno_tensor = torch.tensor(latest_attributes.index.values, dtype=torch.int)

            # Ensure 'date' is also saved as a tensor
            date_tensor = torch.tensor([current_date] * len(latest_attributes), dtype=torch.long)

            # Create graph data object
            graph_data = Data(x=node_features_tensor, edge_index=edge_index, edge_attr=edge_weights, permno=permno_tensor, date=date_tensor)
            filename = f"CorrGraph_{current_date}_W{W}_theta{theta}.pt"
            filepath = os.path.join(save_dir, filename)
            torch.save(graph_data, filepath)
            print(f"Graph for {current_date} saved to {filepath}")
            print(f"Graph for {current_date}: {len(df['PERMNO'].unique())} nodes, {edge_index.size(1) // 2} edges")
        else:
            print(f"Missing 'PERMNO' in latest data for {current_date}")

    # Cleanup to avoid confusion
    df.drop('formatted_date', axis=1, inplace=True)

# Define factor columns
factor_columns =[
    "Beta", "RoE", "InvestPPEInv", "ShareIss5Y", "Accruals", "dNoa",
    "GP", "AssetGrowth", "Investment", "market_cap_adjusted",
    "BM", "CompEquIss", "OperProf", "MaxRet", "IndMom", "DolVol", "Mom1m", "Mom6m", "Mom12m", "Y_excess_return"
]

# Define the return column
return_column = "excess_return"


Call the Function specifying the rolling window size (W), and the cut off.

In [ ]:

# Usage
Ws = [12, 24, 36, 48, 60]
thetas = [0.00001, 0.1, 0.2, 0.3, 0.4, 0.5 , 0.9]
for W in Ws:
    for theta in thetas:
        construct_graph_with_correlation_sign(df, factor_columns, return_column, W, theta)

